In [1]:
import torch
import pandas as pd
import numpy as np
from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import LogExpectedImprovement
from botorch.optim import optimize_acqf

# Generate a sample dataset related to material science properties
np.random.seed(42)

# Let's assume X1 and X2 represent two different processing parameters, such as temperature and pressure
# Y could represent a material property like tensile strength

# Generate random temperature and pressure values
temperature = np.random.uniform(20, 100, 20)  # Temperature in degrees Celsius
pressure = np.random.uniform(1, 10, 20)  # Pressure in MPa

# Let's assume the tensile strength increases with temperature and pressure in a non-linear way
tensile_strength = 10 + 0.5 * temperature + 0.3 * pressure + np.random.normal(0, 1, 20)

# Create a DataFrame
data = pd.DataFrame({
    'Temperature': temperature,
    'Pressure': pressure,
    'TensileStrength': tensile_strength
})

# Save the dataset to a CSV file
data.to_csv('sample_materials_data.csv', index=False)

# Load the data into tensors
X = torch.tensor(data[['Temperature', 'Pressure']].values, dtype=torch.double)
Y = torch.tensor(data['TensileStrength'].values, dtype=torch.double).unsqueeze(-1)

# Initialize the Gaussian Process model with normalization and standardization
gp = SingleTaskGP(
    train_X=X,
    train_Y=Y,
    input_transform=Normalize(d=X.shape[1]),
    outcome_transform=Standardize(m=1),
)

# Fit the model
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

# Define the acquisition function
logNEI = LogExpectedImprovement(model=gp, best_f=Y.max())

# Define the bounds based on the generated data
bounds = torch.stack([X.min(dim=0).values, X.max(dim=0).values]).to(torch.double)

# Optimize the acquisition function
candidate, acq_value = optimize_acqf(
    logNEI,
    bounds=bounds,
    q=1,
    num_restarts=5,
    raw_samples=20,
)

print("Optimized candidate X value:", candidate)
print("Acquisition value:", acq_value)


Optimized candidate X value: tensor([[97.5928,  9.6907]], dtype=torch.float64)
Acquisition value: tensor(-0.2876, dtype=torch.float64)
